In [1]:
library(tidyverse)
library(cytominer)
library(magrittr)
library(RCurl)

-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
<U+221A> ggplot2 2.2.1     <U+221A> purrr   0.2.5
<U+221A> tibble  1.4.2     <U+221A> dplyr   0.7.6
<U+221A> tidyr   0.8.1     <U+221A> stringr 1.3.1
<U+221A> readr   1.1.1     <U+221A> forcats 0.2.0
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()

Attaching package: 'cytominer'

The following object is masked from 'package:stats':

    aggregate

The following object is masked from 'package:base':

    transform


Attaching package: 'magrittr'

The following object is masked from 'package:purrr':

    set_names

The following object is masked from 'package:tidyr':

    extract

Loading required package: bitops

Attaching package: 'RCurl'

The following object is masked from 'package:tidyr':

    complete



In [2]:
load_dataset  <- function(partition, dataset,feature){
    file_name  <- read_csv("../datasets.csv") 
    x  <-  file_name %>% filter(
         Partition == partition,
         Dataset == dataset,
         Features == feature) %>% 
         extract2("Link")

    return(read_csv(x) %>% 
          mutate(Metadata_dataset = dataset) %>%
          mutate(Metadata_partition = partition) %>% 
          mutate(Metadata_features = feature) 
          )
    }

# Load data 
We load training and test datasets for both genetic perturbation experiments 

In [3]:
# bbbc37 data 
bbbc037_train  <- load_dataset("Train","BBBC037","DeepLearning")  %>% 
    mutate(Metadata_x_mutation_status = "none")

bbbc037_test <- load_dataset("Test","BBBC037","DeepLearning")  %>% 
    mutate(Metadata_x_mutation_status = "none")

bbbc037  <- rbind(bbbc037_train, bbbc037_test)

Parsed with column specification:
cols(
  Dataset = col_character(),
  Partition = col_character(),
  Features = col_character(),
  Link = col_character()
)
Parsed with column specification:
cols(
  .default = col_double(),
  Metadata_Well = col_character(),
  Metadata_Plate_Map_Name = col_character(),
  Metadata_well_position = col_character(),
  Metadata_gene_name = col_character(),
  Metadata_pert_name = col_character(),
  Metadata_broad_sample = col_character(),
  Metadata_cell_line = col_character(),
  Metadata_ASSAY_WELL_ROLE = col_character(),
  Metadata_pert_id = col_character(),
  Metadata_pert_mfc_id = col_character(),
  Metadata_pert_well = col_character(),
  Metadata_pert_id_vendor = col_character(),
  Metadata_cell_id = col_character(),
  Metadata_broad_sample_type = col_character(),
  Metadata_pert_type = col_character(),
  Plate_Well = col_character(),
  Metadata_Well.1 = col_character()
)
See spec(...) for full column specifications.
Parsed with column specification:
co

In [4]:
# bbbc043 data 
bbbc043_train  <- load_dataset("Train","BBBC043","DeepLearning")
 
bbbc043_test <- load_dataset("Test","BBBC043","DeepLearning")

bbbc043  <- rbind(bbbc043_train, bbbc043_test)

Parsed with column specification:
cols(
  Dataset = col_character(),
  Partition = col_character(),
  Features = col_character(),
  Link = col_character()
)
Parsed with column specification:
cols(
  .default = col_double(),
  Metadata_Well = col_character(),
  Metadata_Plate_Map_Name = col_character(),
  Metadata_well_position = col_character(),
  Metadata_pert_type = col_character(),
  Metadata_PublicID = col_character(),
  Metadata_Transcript = col_character(),
  Metadata_VirusPlateName = col_character(),
  Metadata_x_mutation_status = col_character(),
  Metadata_broad_sample = col_character(),
  Metadata_pert_name = col_character(),
  Metadata_pert_id = col_character(),
  Metadata_pert_mfc_id = col_character(),
  Metadata_pert_well = col_character(),
  Metadata_pert_id_vendor = col_character(),
  Metadata_cell_id = col_character(),
  Metadata_broad_sample_type = col_character(),
  Metadata_gene_name = col_character()
)
See spec(...) for full column specifications.
Parsed with column

## Check dimensionality

In [85]:
dim(bbbc043)
dim(bbbc037)

[1] 6144 7704

[1] 1915 7705

## Extract common features 

In [91]:
colnames_bbbc037 <- colnames(bbbc037)
colnames_bbbc043 <- colnames(bbbc043)


Metadata_names_bbbc037 <- c(
   stringr::str_subset(colnames_bbbc037, "^Meta")
) 

Metadata_names_bbbc043 <- c(
   stringr::str_subset(colnames_bbbc043, "^Meta")
) 

common_metadata  <- intersect(Metadata_names_bbbc037, Metadata_names_bbbc043)  
common_features  <- setdiff(intersect(colnames_bbbc037, colnames_bbbc043),common_metadata)

bbbc037_na_feature  <- cytominer::drop_na_columns(
    population = bbbc037,
    variables = common_features,
    cutoff = 0
    )

bbbc043_na_feature  <- cytominer::drop_na_columns(
    population = bbbc043,
    variables = common_features,
    cutoff = 0
    )

# Concatenate data sets

In [92]:
population  <- rbind(
    bbbc037 %>% 
        select(c(common_metadata, common_features)),
    bbbc043 %>% 
        select(c(common_metadata, common_features))
    ) %>% 
    mutate(Metadata_perturbation = 'genetic') %>% 
    select(Metadata_perturbation, everything())

## Important: update column names! 

In [93]:
colnames_combined  <- colnames(population)

common_metadata  <- c(
   stringr::str_subset(colnames_combined, "^Meta")
) 

common_features  <- setdiff(colnames_combined, common_metadata)

Cytominer has problems handling column names '1', '2' so we rename them to 'Feature_1', ... 

In [94]:
common_features  <- paste0("Feature_",common_features)
colnames(population)  <- c(common_metadata, common_features)

# Normalize data
We use cytominer to normalize both datasets with respect to the controls, i.e. EMPTY genes

In [95]:
population_normalized  <- cytominer::normalize(
    population, 
    variables = common_features, 
    strata = c("Metadata_perturbation"), 
    sample = population %>% 
                filter(
                    Metadata_gene_name == 'EMPTY',
                    Metadata_partition == "Train"
                ), 
    operation = "standardize"
)

In [96]:
population_normalized %>% dim() %>% print

[1] 8059 7700


# Aggregate data 

In [97]:
population_aggregated  <- cytominer::aggregate(
    population = population_normalized, 
    variables = common_features, 
    strata = c("Metadata_gene_name","Metadata_dataset","Metadata_x_mutation_status"), 
    operation = "mean"
) 

In [98]:
population_normalized %>% extract2("Metadata_gene_name") %>% print

   [1] "VEGFC"      "VHL"        "BMPR1B"     "AXIN2"      "BRCA1"     
   [6] "Luciferase" "CASP9"      "AKT3"       "LacZ"       "TBK1"      
  [11] "CDK2"       "IRAK4"      "CDK4"       "ATM"        "CDKN1A"    
  [16] "ATM"        "MAP3K8"     "EMPTY"      "EMPTY"      "EIF2A"     
  [21] "CTNNB1"     "GSK3B"      "PRKCA"      "MAP3K2"     "CARD11"    
  [26] "CARD11"     "XBP1"       "CDC42"      "HSPA5"      "EMPTY"     
  [31] "VHL"        "GSK3B"      "RAC1"       "EIF4E"      "RAC1"      
  [36] "RB1"        "PRKCE"      "LacZ"       "RHOA"       "CREB1"     
  [41] "DVL1"       "EMPTY"      "NOTCH2"     "PIK3R2"     "MAPK14"    
  [46] "PPP2R5C"    "CSNK1E"     "RELA"       "DVL2"       "ADAM17"    
  [51] "EIF4EBP1"   "EMPTY"      "ELK1"       "TGFBR2"     "FH"        
  [56] "EGLN1"      "Luciferase" "RICTOR"     "IKBKB"      "ACVR1B"    
  [61] "IRS1"       "XIAP"       "TGFBR1"     "CDC42"      "BRAF"      
  [66] "TP53"       "SRC"        "CRY1"       "EMPTY"      "HIF1

In [99]:
population_aggregated %>% slice(1:2) %>% print

# A tibble: 2 x 7,683
  Metadata_gene_n~ Metadata_dataset Metadata_x_muta~ Feature_0 Feature_1
  <chr>            <chr>            <chr>                <dbl>     <dbl>
1 ABCB9            BBBC043          ABCB9_WT.c           0.569     0.601
2 ABCB9            BBBC043          ABCB9_WT.o           0.636     0.578
# ... with 7,678 more variables: Feature_2 <dbl>, Feature_3 <dbl>,
#   Feature_4 <dbl>, Feature_5 <dbl>, Feature_6 <dbl>, Feature_7 <dbl>,
#   Feature_8 <dbl>, Feature_9 <dbl>, Feature_10 <dbl>, Feature_11 <dbl>,
#   Feature_12 <dbl>, Feature_13 <dbl>, Feature_14 <dbl>, Feature_15 <dbl>,
#   Feature_16 <dbl>, Feature_17 <dbl>, Feature_18 <dbl>, Feature_19 <dbl>,
#   Feature_20 <dbl>, Feature_21 <dbl>, Feature_22 <dbl>, Feature_23 <dbl>,
#   Feature_24 <dbl>, Feature_25 <dbl>, Feature_26 <dbl>, Feature_27 <dbl>,
#   Feature_28 <dbl>, Feature_29 <dbl>, Feature_30 <dbl>, Feature_31 <dbl>,
#   Feature_32 <dbl>, Feature_33 <dbl>, Feature_34 <dbl>, Feature_35 <dbl>,
#   Feature_36 <d

# Correlation matrix 

In [100]:
cor_matrix  <- cor(
    x = population_aggregated %>% 
        filter(Metadata_dataset == 'BBBC037') %>% 
        select(common_features) %>% 
        as.matrix() %>% 
        t, 
    y = population_aggregated %>% 
        filter(Metadata_dataset == 'BBBC043') %>% 
        select(common_features) %>% 
        as.matrix() %>% 
        t,
    use  = "complete.obs"
    ) 


# Submision file 

In [101]:
# set column names 
colnames(cor_matrix)  <- population_aggregated %>% 
                            filter(Metadata_dataset == 'BBBC043') %>%
                            extract2("Metadata_x_mutation_status")

# set row names 
rownames(cor_matrix)  <- population_aggregated %>% 
                            filter(Metadata_dataset == 'BBBC037') %>%
                            extract2("Metadata_gene_name")


df  <- cor_matrix %>% as_data_frame() %>% 
            mutate(Metadata_gene_name = population_aggregated %>% 
                            filter(Metadata_dataset == 'BBBC037') %>%
                            extract2("Metadata_gene_name")) %>% 
            select(Metadata_gene_name, everything())

# write submission file
write.csv(df,"../cytodata-baseline_R.csv",row.names = FALSE)